In [ ]:
%%capture
!pip install llama-index==0.10.37 cohere==5.5.0 openai==1.30.1 llama-index-embeddings-openai==0.1.9 qdrant-client==1.9.1 llama-index-vector-stores-qdrant==0.2.8 llama-index-llms-cohere==0.2.0

In [ ]:
import os
import sys
from getpass import getpass
import nest_asyncio

from IPython.display import Markdown, display

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()

sys.path.append('../helpers')

from utils import setup_llm, setup_embed_model, setup_vector_store

In [ ]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

In [ ]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")

In [ ]:
QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

In [ ]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

In [ ]:
from llama_index.core.settings import Settings
from utils import setup_llm, setup_embed_model, setup_vector_store

COLLECTION_NAME = "words-of-the-senpai-naive"

setup_llm(
    provider="cohere", 
    model="command-r-plus", 
    api_key=CO_API_KEY
    )

setup_embed_model(
    provider="openai", 
    model_name="text-embedding-3-large",
    api_key=OPENAI_API_KEY
    )

vector_store = setup_vector_store(QDRANT_URL, QDRANT_API_KEY, COLLECTION_NAME)

In [ ]:
Settings.llm

In [ ]:
Settings.embed_model

# Load Docstore

In [ ]:
from utils import get_documents_from_docstore

documents = get_documents_from_docstore("../data/words-of-the-senpais")

In [ ]:
documents[0].__dict__

# Ingest to VectorDatabase

In [ ]:
from llama_index.core.constants import DEFAULT_CHUNK_SIZE
from llama_index.core.node_parser.text import SentenceSplitter
from llama_index.core import StorageContext
from utils import ingest

print(f"This is the chunk size: {DEFAULT_CHUNK_SIZE}")

tranforms = [
    SentenceSplitter(chunk_size=DEFAULT_CHUNK_SIZE), 
    Settings.embed_model
    ]

nodes = ingest(
    documents=documents,
    transformations=tranforms,
    vector_store=vector_store,
)

# Build index over vector database


In [ ]:
from utils import create_index, create_query_engine

storage_context = StorageContext.from_defaults(
    vector_store=vector_store
    )

index = create_index(
    from_where="vector_store",
    embed_model=Settings.embed_model, 
    vector_store=vector_store, 
    # storage_context=storage_context
    )

query_engine = create_query_engine(
    index=index, 
    mode="query",
    # llm=Settings.llm
    )

# Query Pipelines



In [ ]:
from utils import create_query_pipeline

from llama_index.core.query_pipeline import InputComponent

input_component = InputComponent()

chain = [input_component, query_engine]

query_pipeline = create_query_pipeline(chain)

# Query

In [ ]:
response_1 = query_pipeline.run(input="What mental models, principles or philosophies are most valuable for making important life decisions and navigating an uncertain world?")

In [ ]:
print(response_1)

In [ ]:
response_1.source_nodes[0].__dict__

In [ ]:
response_2 = query_pipeline.run(input="What is the essence of a life well-lived, and what are the key components of a good life that everyone should strive for?")

In [ ]:
response_3 = query_pipeline.run(input="What is the essence of a life well-lived, and what are the key components of a good life that everyone should strive for?")

In [ ]:
response_1 = query_pipeline.run(input="What role does luck play in achieving outsized success and wealth creation? Is capitalizing on luck a skill that can be cultivated?")

In [ ]:
response_1 = query_pipeline.run(input="What are some commonalities between Naval Ravikant and Nassim Taleb's thinking on luck?")